## load model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
with open("hf.token", "r") as f:
    hftoken = f.read().strip()  

import os
cache_dir = "/mnt/c/Users/yc/.cache/huggingface"
os.environ['HF_HOME'] = cache_dir

from huggingface_hub import login
login(token=hftoken)  # Move token to environment variable

from ult import *

/home/yc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
device_map = {"": 0}
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    torch_dtype=torch.bfloat16
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:45<00:00, 26.42s/it]


## data

In [59]:
txt_file_path = 'data/formaldef.txt'
formaldic = txt_to_dict(txt_file_path)
len(formaldic)

meddict={}
for k,v in formaldic.items():
    meddict[k.split('Listen to pronunciation')[0].split('(')[0]]=v
filename= 'data/filtered_medical_dictionary.csv'
eighth_grade_words=set()
with open(filename, 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        if row:  # Make sure row is not empty
            eighth_grade_words.add(row[0])  # Add the word (first column)
filtered_meddict = {word: explanation for word, explanation in meddict.items() 
                   if word in eighth_grade_words}
meddict=filtered_meddict
# load data
df = pd.read_csv('data/CORAL/coral-expert-curated-medical-oncology-reports-to-advance-language-model-inference-1.0/coral/unannotated/data/breastca_unannotated.csv')
# df = df.sample(1, random_state=42)
# df=df.iloc[[44,45,53,70,83, 0,1,2,3,4]] 
df=df.iloc[[44,45,53,70,83]] 
# df=df.iloc[[44,]] 
test_note=df.iloc[0]['note_text']
df

,coral_idx,Sex,UCSFDerivedRaceEthnicity_X,BirthDate,note_text
44,184,Female,Latinx,1983-10-04,Medical Oncology Consult Note Video Consult ...
45,185,Female,Multi-Race/Ethnicity,1973-03-08,This is a shared service. Physician Statement...
53,193,Female,Multi-Race/Ethnicity,1979-06-20,ID: ***** ***** is a 39 y.o. premenopausal pat...
70,210,Female,Southwest Asian and North African,1974-04-05,Patient Name: ***** ***** ***** *****: 08/22/...
83,223,Female,Unknown/Declined,1960-10-12,We performed this consultation using real-time...


# keypoint

In [60]:
extraction_prompts = {

"Reason_for_Visit": """
TASK: Extract 'Reason for Visit'.
Respond *only* with a JSON object using this exact schema:
{
"Patient type": "either New patient or follow up"
"second opinion": "whether the visit is consultation/second opinion or not",
"in-person":" either Televisit or in-person. (note, video consult, televisit, telehealth are the same thing",
"summary": "A brief summary of the reason for visit."
}
""",


"What_We_Found": """
TASK: Extract 'What We Found'. 
Respond *only* with a JSON object using this exact schema:
{
"Type_of_Cancer": "list the type of cancer"
"Stage_of_Cancer": "list the stage if it is written in the note",
"Distant Metastasis": "if there is distant metastasis (met)." "Yes, to where; No, local, not sure, need more evidence such as imaging.",
"Metastasis": "if there is met, Yes (to where), No, or Not sure",
"lab_summary": "Summary of key lab results.",
"findings": "Summary of new findings or disease status."
}
""",


"Treatment_Summary": """
TASK: Extract 'Treatment Summary'. 
Respond *only* with a JSON object using this exact schema:
{
"current_meds": "List of current oncologic medications or regimens.",
"recent_changes": "Any holds, dose reductions, or switches.",
"supportive_meds": "List of supportive medications."
}
""",


"Goals of care": """
TASK: Extract 'What We Discussed / Decided'.
Respond *only* with a JSON object using this exact schema:
{
"goals_of_treatment": "eg, cancer is not curable, but it's treatable and the goal is to extend the duration and maintain the quality of life",
"response_assessment": "How the cancer is responding to the treatment, not responding or responding.",
}
""",


"Medication_Plan": """
TASK: Extract the 'Medication Plan'. Find the 'Assessment/Plan' section of the note, usually the final section.
Include all current and future medication plans for both cancer therapy and supportive treatment. Cancer treatment could be one or many in chemotherapy, hormone therapy, bone therapy, radiotherapy (eg. rad onc, xrt). Supportive treatment could be one or many in bowel regimen, pain medication, psychiatry medication, neuropathy or any blood transfusion plan. 

Include whether a medication is being started now (e.g,“will start”, “Rx sent”, “starting today” ), plan or discuss in the future after certain condition (e.g., “plan to start after radiation”, “discussed addition of…”), continue or maintained(“continue”), stop or change.

Do NOT include procedures, labs, imaging, or genetic testing.
For your information, a completed, finished, status post medication, s/p, means past treatments.
Include an alterative, second-line or clinical trials options if dicussed.

Respond *only* with a JSON object using this exact schema:
{
    "The medication/treatment, one of chemotherapy, hormone therapy, bone therapy, radiotherapy": 
{"summary":"the summary of this type of medication, including start/stop/cotinue if applicable",
"Short term side_effects_discussed": "short term Side effects of this particular medications.",
"Long term side_effects_discussed": "long term Side effects of this particular medications.."}
}
""",

"Medication_Plan": """
TASK: Extract the 'Medication Plan'. Find the 'Assessment/Plan' section of the note, usually the final section.
Include future medication plans, changes to current meds (start, stop, continue), supportive meds, bowel regimen, and any blood transfusion plan.
Respond *only* with a JSON object using this exact schema:
{
    "medication_plan": "A summary of the complete medication plan."
}
""",



"Procedure_Plan": """
TASK: Extract the 'Procedure Plan'. Find the 'Assessment/Plan' section of the note, usually the final section.
Include future procedures including surgery, radiation therapy, or interventional procedures such as biopsy, lumbar puncture, or Chemo Port Insertion. 
Do not include imaging plan, lab plan, or medication plan.
Respond *only* with a JSON object using this exact schema:
{
    "procedure_plan": "A summary of any planned procedures."
}
""",



"Imaging Plan": """
TASK: Extract the 'Imaging Plan'. Find the 'Assessment/Plan' section of the note, usually the final section.
Include all future imaging like CT, MRI, PET/CT, ultrasound,  DEXA scans, including timing and rationale if mentioned.
Do not include any procedure plan, lab plan, or medication plan.
Respond *only* with a JSON object using this exact schema:
{
    "imaging_plan": "A summary of any planned procedures."
}
""",



"Lab Plan": """
TASK: Extract the 'lab Plan'. Find the 'Assessment/Plan' section of the note, usually the final section.
Include future labs like CBC, CMP, tumor markers, coagulation profile, etc. Specify frequency and rationale if mentioned.
Do not include any procedure plan, medication plan, or imagining plan.
Respond *only* with a JSON object using this exact schema:
{
    "lab_plan": "A summary of the future lab plan."
}
""",



"Genetic_Testing_Plan": """
TASK: Extract the 'Genetic Testing Plan'.

1. Scope of section:
- First, find the 'Assessment/Plan' section of the note (usually the final section).
- Only look for plans that are clearly about FUTURE genetic or molecular TESTING.

2. What COUNTS as 'genetic or molecular testing':
- Diagnostic, prognostic, or predictive laboratory assays, such as:
  - Tumor genomic sequencing / NGS / panel testing
  - Germline genetic panels (e.g., hereditary cancer panel, BRCA testing)
  - Liquid biopsy / circulating tumor DNA (ctDNA) tests
  - Specific biomarker tests (e.g., PD-L1 testing, MSI testing, EGFR mutation testing)
  - Any plan to "send", "order", "check", or "obtain" a genetic, genomic, or molecular TEST

3. What MUST be EXCLUDED:
- DO NOT include any medications, systemic therapies, or treatment plans:
  - Chemotherapy, immunotherapy, targeted therapies (e.g., FGFR inhibitor, PARP inhibitor, CDK4/6 inhibitor, TKIs)
  - Hormonal therapy, radiation therapy, surgery
  - Clinical trial options, even if the trial involves targeted drugs or inhibitors
- DO NOT include tests that are already completed, historic, or only mentioned in past oncology history.
- DO NOT include imaging (CT, PET, MRI, X-ray, ultrasound) or routine labs (CBC, CMP, etc.).

4. If there is NO new genetic or molecular test planned in the Assessment/Plan:
- Set the value to a clear negative statement, for example:
  "No new genetic or molecular tests were planned during this visit."

5. Output format:
Respond *only* with a JSON object using this exact schema:
{
    "genetic_testing_plan": "A summary of any future planned genetic or molecular tests, or a clear statement that none are planned."
}
""",




"Referral": """
TASK: Extract 'Referral'. Find the 'Assessment/Plan' section of the note, usually the final section.

Respond *only* with a JSON object using this exact schema:
{
"Nutrition": "any nutration referrals such as diet optimization, appetite, weight maintenance), "Genetics": "eg, germline testing, family counseling",
"Specialty": "eg, Palliative care (symptom or pain management, goals of care), Radiation oncology, surgical oncology, Psychology, psychiatry for coping and mood support"
"Others": "Physical or occupational therapy, Social work, financial counseling}
""",



"follow up/next visit": """
TASK: Extract the 'follow up/next visit'. Find the 'Assessment/Plan' section of the note, usually the final section.

Respond *only* with a JSON object using this exact schema:
{
    "Next clinic visit": "(in-person or telehealth): timing and purpose"
}
""",



"Advance care planning": """
TASK: Extract the 'Advance care planning'.
Include Advance directives, health-care proxy, code status (if appropriate)、
Respond *only* with a JSON object using this exact schema:
{
    "Advance care": "A summary of any planned Advance care."
}
""",

}



In [61]:
# res=[]

# for row in df.itertuples():
#     test_note=row.note_text
#     gen_config = {
#     "max_new_tokens": 512,  # <-- Increased from 150 to 512
#     "eos_token_id": tokenizer.eos_token_id,
#     "do_sample": False
#     }

#     # myprint("\n--- 1. Creating Base KV Cache from long text... ---")

#     base_prompt = (
#         f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
#         f"You are a medical data extraction expert. You will be given a long medical note. "
#         f"Your task is to answer a series of questions about it, one by one. "
#         # FIX 2: Stricter instruction in system prompt
#         f"Respond *only* with the valid JSON object requested. Do not add markdown backticks or any other text."
#         f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
#         f"Here is the medical note:\n\n"
#         f"--- BEGIN NOTE ---\n{test_note}\n--- END NOTE ---"
#         f"\n\nI will now ask you to extract specific sections. "
#         f"Please wait for my first extraction task."
#         f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
#         f"{{\"status\": \"Understood. I have read the note and am ready.\"}}"
#     )

#     # myprint("  Tokenizing and running forward pass to get base cache...")
#     with torch.no_grad():
#         inputs = tokenizer(base_prompt, return_tensors="pt").to(model.device)
        
#         # This logic to create the base_cache is correct
#         outputs = model(
#             input_ids=inputs["input_ids"], 
#             attention_mask=inputs["attention_mask"],
#             use_cache=True
#         )
        
#         base_cache = outputs.past_key_values
        
#         # Clean up
#         del inputs, outputs
#         torch.cuda.empty_cache()
#         gc.collect()

#     # myprint("Base KV Cache created. Ready for 'branching' extractions.")

#     # --- FIX 3: Define STRICT SCHEMAS for each extraction task ---
#     # This is the most important "prompt engineering" fix.
#     # We are telling the model *exactly* what keys to use.


#     extracted_data = {}

#     # myprint("\n--- 2. Running EFFICIENT 'Branching' Extractions ---")

#     # (Assuming `run_model_with_cache_manual` is your function `run_model_with_cache`)
#     # If not, please rename this call to `run_model_with_cache`
#     run_model_function = run_model_with_cache_manual 

#     for key, task in extraction_prompts.items():
#         # myprint(f"\nExtracting: {key}...")
        
#         # --- FIX 4: Remove the BOS token from the loop ---
#         # The base_cache *already* contains the BOS token.
#         # Adding it again can cause errors.
#         task_prompt = (
#             f"<|start_header_id|>user<|end_header_id|>\n\n" # <-- Removed <|begin_of_text|>
#             f"{task}" # <-- Use the new, detailed prompt
#             f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
#         )

#         # Use the SAME base_cache for each extraction
#         answer, returned_cache = run_model_function(
#             task_prompt, 
#             model, 
#             tokenizer, 
#             gen_config, 
#             kv_cache=base_cache
#         )
        
#         # Your memory management here is good.
#         del returned_cache
#         torch.cuda.empty_cache()
#         gc.collect()
        
#         # myprint(f"  Raw Output: {answer}")
        
#         try:
#             # We are now *only* expecting a JSON object
#             clean_answer = answer.strip().strip("```json").strip("```").strip()
#             extracted_data[key] = json.loads(clean_answer)
#         except json.JSONDecodeError:
#             extracted_data[key] = {"error": "Failed to parse JSON", "raw": answer}

#     # myprint("\n--- 3. All extractions complete. ---")
#     # myprint("\n--- FINAL EXTRACTED DATA ---")
#     res.append(json.dumps(extracted_data, indent=2))
#     keypoints=res

In [62]:
# resdf=[json.loads(r) for r in res]
# for i in range(1):
#     test_note=df.iloc[i]['note_text']
#     resdf[i]['note']=test_note
#     resdf[i]['coral_idx']=df.iloc[i].coral_idx
# resdf=pd.DataFrame(resdf)
# resdf

In [63]:
# for i, r in enumerate(res):
#     coral_idx = df.iloc[i].coral_idx
#     print("=" * 30)
#     print(f"\ncoral_idx: {coral_idx}\n")
#     print(r)

In [ ]:

def create_explanation_prompt(note_text: str, keypoints: str) -> str:


    system_prompt = f"""
### Persona
You are an experienced and compassionate Oncologist and medical educator. Your primary role is to translate complex medical information into clear, 8th-grade level English. Your tone should be professional, empathetic, and honest.

### Golden Rule: Radical Simplicity - Translate, Don't Transfer
Your single most important task is to convert medical terminology into simple explanations. Do not just define a term; replace it with an easy-to-understand explanation.

### Your Task:
Write a complete letter explaining the information from the medical text. Imagine you are sitting with the patient and explaining this to them in person.

### Letter Structure and Flow:
Organize your letter logically using the following question-based headers:
* **Empathetic Opening:** Start with a warm and supportive salutation.
* **Why you came today:** Briefly explain the purpose of the visit in plain words.
* **What you told us:** Summarize what the patient and family shared (symptoms, worries, questions).
* **What did we find** Explain what the team learned from today's visit (exam, blood work, scans) in simple terms. you need to include all 'key points' mentioned.
* **What is the plan** List the next steps (treatment, monitoring, follow-up) in plain language. refer to the keypoints
* **Closing with Support:** End by reinforcing your support. Limit to one sentence maximum.

### STRICT NEGATIVE CONSTRAINTS:
* Do not say anything not present in the medical note.
* If cancer has spread (metastasis), DO NOT list the specific organs affected. Say "the cancer has spread to other parts of the body."
* Do not use fatalistic language. The focus MUST be on quality of life.

### Strict Output Formatting:
Provide ONLY the letter. Your output must start directly with "Dear [Patient Name]," and end with "Sincerely,\nYour Care Team at [Institution]". stop generating after that.

"""
    
    user_prompt = f"""
### Original Medical Text:
<begin_of_note_text> 
"{note_text}"
<end_of_note_text> 

### key points that should be included in the letter:
<begin_of_keypoints>
"{keypoints}"
<end_of_keypoints>

"""

    return user_prompt+system_prompt



def create_cleaning_prompt(raw_response: str) -> str:

    system_prompt ='''
    ### Your Task
Revise the provided medical_text.
1.  **Clean Up Language:** Improve sentence structure and use professional language.
2.  **Improve Flow:** Enhance the logical flow and transitions.
3.  Remove formatting subtitles, remove additional notes after the letter.
    '''
    user_prompt = f"""

### Medical Text to Revise:

<begin_of_medical_text>
{raw_response}
<end_of_medical_text>

now, begin your revision. do not add any additional note after your revised letter.
"""

    return system_prompt+user_prompt


terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


EXPLANATION_GENERATION_CONFIG = {
    "max_new_tokens": 666,
    "temperature": 0.3,
    "top_p": 0.85,
    "repetition_penalty": 1.2,     
    "no_repeat_ngram_size": 3,     
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": terminators,
    "early_stopping": True,
}

CLEANING_CONFIG = {
    "max_new_tokens": 666,
    "temperature": 0.1,
    "top_p": 0.9,
    "repetition_penalty": 1.2,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": terminators, 
    "early_stopping": True,
}

KEYPOINT_CONFIG = {
    "max_new_tokens": 512, 
    "eos_token_id": tokenizer.eos_token_id,
    "do_sample": False
    }

In [75]:
all_results = {}

for index, row in df.iterrows():
    note_text = row['note_text'] 
    print(f"\nProcessing row {index + 1}/{len(df)}...")

    # get summary
    keypoints_base_prompt = (
        f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n"
        f"You are a medical data extraction expert. You will be given a long medical note. "
        f"Your task is to answer a series of questions about it, one by one. "
        # FIX 2: Stricter instruction in system prompt
        f"Respond *only* with the valid JSON object requested. Do not add markdown backticks or any other text."
        f"<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
        f"Here is the medical note:\n\n"
        f"--- BEGIN NOTE ---\n{note_text}\n--- END NOTE ---"
        f"\n\nI will now ask you to extract specific sections. "
        f"Please wait for my first extraction task."
        f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        f"{{\"status\": \"Understood. I have read the note and am ready.\"}}"
    )
    with torch.no_grad():
        inputs = tokenizer(keypoints_base_prompt, return_tensors="pt").to(model.device)
        outputs = model(
            input_ids=inputs["input_ids"], 
            attention_mask=inputs["attention_mask"],
            use_cache=True
        )
        base_cache = outputs.past_key_values
        del inputs, outputs
        torch.cuda.empty_cache()
        gc.collect()
    keypoints = {}
    run_model_function = run_model_with_cache_manual 
    for key, task in extraction_prompts.items():
        task_prompt = (
            f"<|start_header_id|>user<|end_header_id|>\n\n" 
            f"{task}"
            f"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n"
        )
        answer, returned_cache = run_model_function(
            task_prompt, 
            model, 
            tokenizer, 
            KEYPOINT_CONFIG, 
            kv_cache=base_cache
        )
        del returned_cache
        torch.cuda.empty_cache()
        gc.collect()
        
        try:
            clean_answer = answer.strip().strip("```json").strip("```").strip()
            keypoints[key] = json.loads(clean_answer)
        except json.JSONDecodeError:
            keypoints[key] = answer

    print('keypoints')

    print('explain')
    explanation_prompt= create_explanation_prompt(note_text, keypoints)
    explanation,_  = run_model_function(explanation_prompt, model, tokenizer, EXPLANATION_GENERATION_CONFIG)

    # clean up
    print('clean')
    cleaning_prompt = create_cleaning_prompt(explanation)
    final_result,_  = run_model_function(cleaning_prompt, model, tokenizer, CLEANING_CONFIG)
    
    # # --- Store and Print Results ---
    row_result = {
        'coral_idx': row['coral_idx'],
        # 'note_text': note_text,
        'keypoints': keypoints,
        'raw_result': explanation,
        'clean_result': final_result
    }
    
    all_results[index]=(row_result)
    print(f"\n{'='*60}")
    print(f"RESULTS FOR ROW {index + 1}")
    print(f"{'='*60}")
    
    for col in ['keypoints', 'raw_result', 'clean_result']:
        print(f"\n--- Column: {col} ---")
        original_text = row_result[col]
        print(original_text)


Processing row 45/5...
keypoints
explain
clean

RESULTS FOR ROW 45

--- Column: keypoints ---
{'Reason_for_Visit': {'Patient type': 'New patient', 'second opinion': 'Yes', 'in-person': 'Televisit', 'summary': 'Patient presents with new patient evaluation'}, 'What_We_Found': {'Type_of_Cancer': 'Malignant neoplasm of overlapping sites of right breast in female, estrogen receptor negative', 'Stage_of_Cancer': 'Stage IIIB', 'Distant Metastasis': 'Yes, to the lung and right hilar lymph node', 'Metastasis': 'Yes, to the lung and right hilar lymph node', 'lab_summary': 'No lab results mentioned', 'findings': 'Metastatic triple negative breast cancer to the lung and right hilar lymph node'}, 'Treatment_Summary': {'current_meds': 'Xeloda', 'recent_changes': 'Started on taxol, but did not complete due to progression', 'supportive_meds': 'Reishi mushroom, Papaya enzymes, Mullein extract'}, 'Goals of care': {'goals_of_treatment': 'We discussed how metastatic breast cancer is not curable but is tr